In [1]:
import os
import re
import supervisely as sly
from mmengine.config import Config
from mmdet3d.apis import LidarDet3DInferencer
from src.tests.extract_weights_url import find_weights_url
from src.sly_utils import download_point_cloud, upload_point_cloud
from src.inference.pcd_inferencer import PcdDet3DInferencer
from src.inference.functional import create_sly_annotation, up_bbox3d, filter_by_confidence
from src.pcd_utils import convert_bin_to_pcd


# globals    
api = sly.Api()
project_id = 32768
dataset_id = 81541
project_meta = sly.ProjectMeta.from_json(api.project.get_meta(project_id))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [2]:
def get_last_pth_weights():
    base_path = "app_data/work_dir/"
    weights = [os.path.join(base_path, f) for f in os.listdir(base_path) if f.endswith(".pth")]
    return max(weights, key=os.path.getctime)

def get_last_config():
    base_path = "app_data/work_dir/"
    # get last dir in work_dir by name
    dirs = [os.path.join(base_path, f) for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    dir = sorted(dirs)[-1]
    config = os.path.join(dir, "vis_data", "config.py")
    return config

In [12]:
# Model
weights_url = "app_data/work_dir/epoch_10-pointpillars.pth"
cfg_model = "app_data/work_dir/20231215_125044-pointpillars/vis_data/config.py"

weights_url = "app_data/work_dir/epoch_20-centerpoint.pth"
cfg_model = "app_data/work_dir/20231214_142029-centerpoint/vis_data/config.py"

weights_url = get_last_pth_weights()
cfg_model = get_last_config()

print("weights_url: ", weights_url)
print("cfg_model: ", cfg_model)

import datetime
time = datetime.datetime.fromtimestamp(os.path.getctime(weights_url))
print("weights_url created: ", time)

# model_index = "mmdetection3d/model-index.yml"
# weights_url = find_weights_url(model_index, re.sub("_custom.*\.py", ".py", cfg_model))

weights_url:  app_data/work_dir/epoch_40.pth
cfg_model:  app_data/work_dir/20240108_001510/vis_data/config.py
weights_url created:  2024-01-08 00:28:10.092466


In [4]:
# To import projects
import os, sys
sys.path.append(os.path.abspath("mmdetection3d"))

In [15]:
# 4 cars
# pcd_path = "app_data/lyft/LYFT/pointcloud/host-a005_lidar1_1231201454801395736.pcd"
# many cars
pcd_path = "app_data/lyft/LYFT/pointcloud/host-a005_lidar1_1231201437602160096.pcd"

_, ext = os.path.splitext(pcd_path)
is_bin = ext == ".bin"

# Make config
# loading existed 'cfg_model' is unsafe, because of inappropriate pipelines
cfg = Config.fromfile(cfg_model)
model_class_names = cfg.class_names
print(f"Model class names: {model_class_names}")

# add classes to project meta
need_update = False
for class_name in model_class_names:
    if project_meta.get_obj_class(class_name) is None:
        from supervisely.geometry.cuboid_3d import Cuboid3d
        project_meta = project_meta.add_obj_class(sly.ObjClass(class_name, Cuboid3d))
        print(f"Added class {class_name} to project meta.")
        need_update = True
if need_update:
    api.project.update_meta(project_id, project_meta.to_json())
    api.project.pull_meta_ids(project_id, project_meta)

# Inference
if is_bin:
    inferencer = LidarDet3DInferencer(cfg_model, weights_url, device='cuda:0')
else:
    inferencer = PcdDet3DInferencer(cfg_model, weights_url, device='cuda:0')

results_dict = inferencer(inputs=dict(points=pcd_path), no_save_vis=True)

predictions = results_dict['predictions'][0]
bboxes_3d = predictions['bboxes_3d']
labels_3d = predictions['labels_3d']
scores_3d = predictions['scores_3d']
bboxes_3d, labels_3d, scores_3d = filter_by_confidence(bboxes_3d, labels_3d, scores_3d, threshold=0.45)
bboxes_3d = [up_bbox3d(bbox3d) for bbox3d in bboxes_3d]

# Create annotation
ann = create_sly_annotation(bboxes_3d, labels_3d, model_class_names, project_meta)

# Upload pcd
if is_bin:
    convert_bin_to_pcd(pcd_path, "tmp.pcd")
    pcd_path = "tmp.pcd"
name = "tmp_infer_"+sly.rand_str(8)+".pcd"
pcd_info = upload_point_cloud(api, dataset_id, pcd_path, name=name)

# Upload annotation
pcd_id = pcd_info.id
api.pointcloud.annotation.append(pcd_id, ann)

print(name)
print(f"https://dev.supervise.ly/app/point-clouds/?datasetId={dataset_id}&pointCloudId={pcd_id}")


Output()

Model class names: ['car', 'pedestrian', 'truck']
Loads checkpoint by local backend from path: app_data/work_dir/epoch_40.pth
self.METAINFO['classes']=['car', 'pedestrian', 'truck'], self.label_mapping={0: 0, 1: 1, 2: 2, -1: -1}


TypeError: list indices must be integers or slices, not str

In [16]:
inferencer._init_pipeline(cfg)

Compose(
    MultiScaleFlipAug3D(transforms=Compose(
    GlobalRotScaleTrans(rot_range=[0, 0], scale_ratio_range=[1.0, 1.0], translation_std=[0, 0, 0], shift_height=False)
    RandomFlip3D(sync_2d=True, flip_ratio_bev_vertical=0.0)
    PointsRangeFilter(point_cloud_range=[0.0, -10.0, -3.0, 10.399999618530273, 10.0, 1.0])
), img_scale=[(1333, 600)], flip=False, pts_scale_ratio=[1.0], flip_direction=['horizontal'])
    PointsRangeFilter(point_cloud_range=[0.0, -10.0, -3.0, 10.399999618530273, 10.0, 1.0])
    Pack3DDetInputs(keys=['points'])(meta_keys=('img_path', 'ori_shape', 'img_shape', 'lidar2img', 'depth2img', 'cam2img', 'pad_shape', 'scale_factor', 'flip', 'pcd_horizontal_flip', 'pcd_vertical_flip', 'box_mode_3d', 'box_type_3d', 'img_norm_cfg', 'num_pts_feats', 'pcd_trans', 'sample_idx', 'pcd_scale_factor', 'pcd_rotation', 'pcd_rotation_angle', 'lidar_path', 'transformation_3d_flow', 'trans_mat', 'affine_aug', 'sweep_img_metas', 'ori_cam2img', 'cam2global', 'crop_offset', 'img_crop_